In [ ]:
from bs4 import BeautifulSoup
import requests

def scrape_all_page(url):
    all_articles = []  # Liste des articles collectés
    seen_articles = set()  # Ensemble pour éviter les doublons d'URL

    try:
        # Envoyer une requête HTTP pour récupérer la page
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Trouver tous les articles via la classe "articlePreview-card"
            articles = soup.find_all("article", class_="articlePreview-card")
            
            for article in articles:
                # Extraire le lien de l'article
                link_element = article.find("a", class_="item-image")
                article_url = f"https://www.huffingtonpost.fr{link_element['href']}" if link_element else ""

                # Extraire le titre
                title_element = article.find("div", class_="title")
                title_text = title_element.find("h2").get_text(strip=True) if title_element else ""

                # Extraire la description
                description_element = article.find("div", class_="articlePreview-chapo")
                description_text = description_element.get_text(strip=True) if description_element else ""

                # Vérifier si l'article est déjà collecté
                if article_url and article_url not in seen_articles:
                    all_articles.append([title_text, description_text, article_url])
                    seen_articles.add(article_url)

            return all_articles

        else:
            print(f"Erreur HTTP {response.status_code} lors de l'accès à {url}")
            return []

    except requests.RequestException as e:
        print(f"Erreur de requête pour {url}: {e}")
        return []


In [ ]:
import requests
from bs4 import BeautifulSoup

# Fonction pour extraire les informations d'un article
def fetch_article_details(article_data, headers):
    title, summary, url = article_data
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extraire le titre
            title_tag = soup.find('h1', class_='article__title')
            title = title_tag.get_text(strip=True) if title_tag else "Titre non trouvé"

            # Extraire la description (résumé)
            summary_tag = soup.find('p', class_='article__desc')
            summary = summary_tag.get_text(strip=True) if summary_tag else "Résumé non trouvé"

            # Extraire la date
            date_tag = soup.find("span", class_="article-metas__date")
            date = date_tag.get_text(strip=True) if date_tag else "Date non trouvée"

            # Extraire les paragraphes à partir de la classe 'article__paragraph'
            paragraphs = soup.find_all('p', class_='article__paragraph')
            paragraph_list = [p.get_text(strip=True) for p in paragraphs]

            return {
                "Title": title,
                "Summary": summary,
                "Date": date,
                "Paragraphs": paragraph_list
            }
        else:
            print(f"Erreur HTTP {response.status_code} pour {url}")
            return None
    except requests.RequestException as e:
        print(f"Erreur de requête pour {url}: {e}")
        return None

# Exemple d'utilisation
article_data = ("Titre initial", "Résumé initial", "https://url-de-l-article.com")
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

article_details = fetch_article_details(article_data, headers)
print(article_details)


In [ ]:
def filter_articles_with_bio(dataset):
    filtered_articles = []
    total_articles = len(dataset)
    
    # Initialiser tqdm pour la boucle avec un calcul du temps estimé
    start_time = time.time()  # Démarrer le chronomètre
    
    for i, article in tqdm(enumerate(dataset), total=total_articles, desc="Filtrage des articles", unit="article"):
        # Vérifier si le mot "bio" est dans le titre ou le résumé
        if contains_bio(article["Title"]) or contains_bio(article["Summary"]):
            filtered_articles.append(article)
        else:
            # Vérifier chaque paragraphe si le mot "bio" est présent
            for paragraph in article["Paragraphs"]:
                if contains_bio(paragraph):
                    filtered_articles.append(article)
                    break
        
        # Calculer et afficher le temps estimé restant
        elapsed_time = time.time() - start_time  # Temps écoulé
        progress = (i + 1) / total_articles  # Progression
        estimated_time_left = elapsed_time / progress - elapsed_time  # Temps estimé restant
        print(f"\rProgression : {progress * 100:.2f}% - Temps restant estimé : {estimated_time_left / 60:.2f} minutes", end="")
    
    print()  # Pour un retour à la ligne après l'affichage du progrès
    return filtered_articles